In [25]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

In [3]:
quote_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
#query the website and return the html to the variable ‘page’
page = urlopen(quote_page)
soup = BeautifulSoup(page, 'html.parser') #store in variable `soup`

In [5]:
#extract table and convert into dataframe
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df=pd.DataFrame(df)
header = df.iloc[0]
df = df[1:]
df = df.rename(columns = header)
df.head(15)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [6]:
# rows wich has duplicate value of Postcode will be combined into one row
#with the neighborhoods separated with a comma 
df_new = df.groupby(['Postcode','Borough'], sort=False)['Neighbourhood'].apply(', '.join).reset_index()

In [7]:
#change value of Neighbourhood: if Borough isn't 'Not assigned', Neighbourhood = Borough  
for i in range(0, len(df_new)):
    if df_new['Neighbourhood'].iloc[i] =='Not assigned':
        df_new['Neighbourhood'].values[i] = df_new['Borough'].values[i]

In [8]:
df_new.head(1) #show new dataframe

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned


In [9]:
import geocoder 
import folium

In [10]:
df_new['Latitude'] = 0
df_new['Longitude'] = 0

In [11]:
import geopy
from geopy.geocoders import Nominatim

In [15]:
df_new = df_new[df_new['Borough']!='Not assigned']

In [22]:
for i in range(0, len(df_new)):
    address = df_new['Borough'].iloc[i]
    #print(address)
    geolocator = Nominatim()
    location = geolocator.geocode(address, timeout=100)
    latitude= location.latitude
    longitude= location.longitude
    df_new['Latitude'].iloc[i] = latitude
    df_new['Longitude'].iloc[i] = longitude

C:\Users\becchettivi\AppData\Local\Continuum\anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning
C:\Users\becchettivi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
df_new.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.770916,-79.412410
3,M4A,North York,Victoria Village,43.770916,-79.412410
4,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654027,-79.380200
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.770916,-79.412410
6,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
8,M9A,Etobicoke,Islington Avenue,43.643556,-79.565633
9,M1B,Scarborough,"Rouge, Malvern",54.284760,-0.409034
11,M3B,North York,Don Mills North,43.770916,-79.412410
12,M4B,East York,"Woodbine Gardens, Parkview Hill",43.691339,-79.327821
13,M5B,Downtown Toronto,"Ryerson, Garden District",43.654027,-79.380200
